In [1]:
import os
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
dataCSV = os.path.join(os.getcwd(), "..", "CSGODemo", "Dataset", "RWS-CSGO-Dataset.csv")
modelSaveFolder = os.path.join(os.getcwd(), "modelSave")

df = pd.read_csv(dataCSV)

print(df.columns.tolist())

invalidRows = (df == -1).any(axis=1)
pd.set_option('display.max_columns', None)

print(f"Invalid rows --> {invalidRows}")

print(df.isna())

['Player_Kills', 'Player_Assists', 'Player_Deaths', 'Player_HeadshotsPerc', 'Player_KR_Ratio', 'Player_KD_Ratio', 'Player_KAST', 'Player_EntryKills', 'Player_RWS', 'Player_Rating', 'Player_Rating2', 'Player_ATD', 'Player_ADR', 'Player_ELO', 'Player_matchWinPerc', 'Player_RoundWinPerc', 'Team_Player1_ELO', 'Team_Player2_ELO', 'Team_Player3_ELO', 'Team_Player4_ELO', 'Team_Average_ELO', 'Enemy_Player1_ELO', 'Enemy_Player2_ELO', 'Enemy_Player3_ELO', 'Enemy_Player4_ELO', 'Enemy_Player5_ELO', 'Enemy_Average_ELO', 'Map', 'RWS_Target']
Invalid rows --> 0         False
1         False
2         False
3         False
4         False
          ...  
104287    False
104288    False
104289    False
104290    False
104291    False
Length: 104292, dtype: bool
        Player_Kills  Player_Assists  Player_Deaths  Player_HeadshotsPerc  \
0              False           False          False                 False   
1              False           False          False                 False   
2             

In [3]:
abs_correlations = df.corr()[['RWS_Target']].abs()

# Sort absolute correlations in descending order
sorted_abs_correlations = abs_correlations.sort_values(by='RWS_Target', ascending=False)

print(sorted_abs_correlations.head(30))

                      RWS_Target
RWS_Target              1.000000
Player_ADR              0.355642
Player_KD_Ratio         0.350668
Player_RWS              0.347096
Player_Kills            0.344239
Player_Rating2          0.342585
Player_Rating           0.342190
Player_KR_Ratio         0.321415
Player_KAST             0.287832
Player_ELO              0.245620
Player_EntryKills       0.184945
Player_ATD              0.159909
Player_RoundWinPerc     0.148778
Player_matchWinPerc     0.118596
Player_Assists          0.106233
Team_Average_ELO        0.104212
Player_HeadshotsPerc    0.101415
Enemy_Average_ELO       0.091263
Player_Deaths           0.084233
Enemy_Player5_ELO       0.066967
Enemy_Player4_ELO       0.063226
Enemy_Player3_ELO       0.050809
Enemy_Player1_ELO       0.047728
Enemy_Player2_ELO       0.042164
Team_Player4_ELO        0.028214
Team_Player1_ELO        0.013386
Team_Player3_ELO        0.011965
Team_Player2_ELO        0.007301


C:\Users\S_CSIS-PostGrad\AppData\Local\Temp\ipykernel_27952\272354080.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  abs_correlations = df.corr()[['RWS_Target']].abs()


In [4]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(sparse=False, drop='first')
map_column = ['Map']
selected_feature_columns = [
    'Player_ADR', 'Player_KD_Ratio', 'Player_Rating2', 'Player_RWS', 
    'Player_Kills', 'Player_Rating', 'Player_KR_Ratio', 'Player_ELO', 
    'Player_KAST', 'Player_RoundWinPerc', 'Player_matchWinPerc', 
    'Enemy_Average_ELO', 'Team_Average_ELO', 'Player_EntryKills', 'Player_ATD',
]

features = df[selected_feature_columns]
labels = df['RWS_Target']
encoded_maps = onehot_encoder.fit_transform(df[map_column])
encoded_maps_df = pd.DataFrame(encoded_maps, columns=onehot_encoder.get_feature_names_out(map_column))
features_encoded = pd.concat([features, encoded_maps_df], axis=1)


print(features_encoded.shape)
print(labels.shape)
print(features_encoded.columns)



(104292, 27)
(104292,)
Index(['Player_ADR', 'Player_KD_Ratio', 'Player_Rating2', 'Player_RWS',
       'Player_Kills', 'Player_Rating', 'Player_KR_Ratio', 'Player_ELO',
       'Player_KAST', 'Player_RoundWinPerc', 'Player_matchWinPerc',
       'Enemy_Average_ELO', 'Team_Average_ELO', 'Player_EntryKills',
       'Player_ATD', 'Map_de_ancient', 'Map_de_anubis', 'Map_de_cache',
       'Map_de_dust2', 'Map_de_inferno', 'Map_de_mirage', 'Map_de_nuke',
       'Map_de_overpass', 'Map_de_train', 'Map_de_tuscan', 'Map_de_vertigo',
       'Map_workshop/1542127528/de_cbble'],
      dtype='object')


c:\Users\S_CSIS-PostGrad\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_encoded_features = scaler.fit_transform(features_encoded)

X_train, X_test, y_train, y_test = train_test_split(scaled_encoded_features, labels, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(83433, 27)
(20859, 27)
(83433,)
(20859,)


In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

reg = RandomForestRegressor(n_estimators=100, random_state=42, verbose=2)
reg.fit(X_train, y_train)

y_pred = reg.predict(X_test)

building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  3.0min


building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.2s


In [7]:
mse = mean_squared_error(y_test, y_pred)

print(f"Mean Squared Error --> {mse}")

Mean Squared Error --> 11.661866629288077


In [42]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [48]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(X_train.shape[1],)))

    # Vary the number of hidden layers
    for i in range(hp.Int('num_hidden_layers', min_value=1, max_value=5)):
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=128, step=32),
                                       activation=hp.Choice('activation_' + str(i), values=['relu', 'tanh', 'sigmoid']),
                                       kernel_regularizer=tf.keras.regularizers.l2(hp.Choice('l2_reg_' + str(i), values=[1e-3, 1e-4, 1e-5]))))

    model.add(tf.keras.layers.Dense(1))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-4])),
                  loss='mean_squared_error')

    return model
    


In [49]:
tuner = RandomSearch(build_model, objective='val_loss', max_trials=100, executions_per_trial=3, directory='modelSave', project_name = 'CSGOModel')

tuner.search(train_dataset, epochs=100, validation_data = test_dataset)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

best_model = tuner.hypermodel.build(best_hps)

Trial 7 Complete [00h 05m 16s]
val_loss: 13.49855899810791

Best val_loss So Far: 11.716382344563803
Total elapsed time: 00h 33m 03s

Search: Running Trial #8

Value             |Best Value So Far |Hyperparameter
4                 |1                 |num_hidden_layers
128               |96                |units_0
relu              |sigmoid           |activation_0
0.001             |1e-05             |l2_reg_0
0.0001            |0.01              |learning_rate
96                |None              |units_1
relu              |None              |activation_1
1e-05             |None              |l2_reg_1
128               |None              |units_2
sigmoid           |None              |activation_2
0.001             |None              |l2_reg_2
128               |None              |units_3
sigmoid           |None              |activation_3
1e-05             |None              |l2_reg_3

Epoch 1/100
1149/1149 [==============================] - 2s 1ms/step - loss: 29.6662 - val_loss: 17.30